In [2]:

import torch
from transformers import AutoModel, AutoTokenizer,BitsAndBytesConfig
from peft import PeftModel,PeftConfig
base_model_path="/mnt/data/chatglm3-6b-model"
q_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
base_model=AutoModel.from_pretrained(base_model_path,quantization_config=q_config,device_map="auto",trust_remote_code=True)


Loading checkpoint shards: 100%|██████████| 7/7 [00:13<00:00,  1.92s/it]


In [4]:
tokenizer=AutoTokenizer.from_pretrained(base_model_path,trust_remote_code=True)


In [6]:
ft_model_path="./phb/chatglm3-ft"
config=PeftConfig.from_pretrained(ft_model_path)
model=PeftModel.from_pretrained(base_model,ft_model_path)

def compare_base_to_ft(q):
    base_response,base_history=base_model.chat(tokenizer,q)

    inputs=tokenizer(q,return_tensors="pt").to(0)
    ft_outputs=model.generate(**inputs)
    ft_response=tokenizer.decode(ft_outputs[0],skip_special_tokens=True)
    print("问题：{}\n".format(q))
    print("basic：{}".format(base_response))
    print("ft:"+ft_response)

In [7]:
compare_base_to_ft("今天过得怎样？")

2024-01-30 19:17:36.045974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:17:36.468002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 19:17:36.468038: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 19:17:36.470884: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 19:17:36.706271: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-30 19:17:36.708346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

问题：今天过得怎样？

basic：作为一款人工智能助手，我没有个人感受和经历，但我一直处于准备状态，随时为您提供帮助和服务。如果您有什么需要帮助的问题，请随时告诉我，我会尽力为您提供解答和解决方案。
ft:[gMASK]sop 今天过得怎样？ 亲爱的，今天我过得很好，谢谢你的关心。你呢


## 问题：今天过得怎样？

#basic：作为一款人工智能助手，我没有个人感受和经历，但我一直处于准备状态，随时为您提供帮助和服务。如果您有什么需要帮助的问题，请随时告诉我，我会尽力为您提供解答和解决方案。

ft:[gMASK]sop 今天过得怎样？ 亲爱的，今天我过得很好，谢谢你的关心。你呢